## import modules

In [1]:
# Import basic modules
import math
import os
import sys
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore") # Ignore some annoying warnings
from datetime import datetime

# Import module for EMD decomposition
# It is the EMD-signal module with different name to import
from PyEMD import EMD,EEMD,CEEMDAN,Visualisation #For module 'PyEMD', please use 'pip install EMD-signal' instead.

# Import modules for LSTM prediciton
# Sklearn
from sklearn.preprocessing import MinMaxScaler # Normalization
from sklearn.metrics import r2_score # R2
from sklearn.metrics import mean_squared_error # MSE
from sklearn.metrics import mean_absolute_error # MAE
from sklearn.metrics import mean_absolute_percentage_error # MAPE


# Statistical tests
from statsmodels.tsa.stattools import adfuller # adf_test
from statsmodels.stats.diagnostic import acorr_ljungbox as lb_test # LB_test
from statsmodels.stats.stattools import jarque_bera as jb_test # JB_test
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf # plot_acf_pacf

from sklearn.preprocessing import StandardScaler

import yfinance as yf

from xgboost import XGBRegressor as XGBR

In [2]:
data = yf.download("^HSI", start="2017-07-26", end="2022-07-26")

[*********************100%***********************]  1 of 1 completed


In [3]:
data = data.iloc[:,[0,1,2,3,5]]
#data.index = range(len(data))

In [4]:
data.head()

,Open,High,Low,Close,Volume
Date,,,,,
2017-07-26,26960.810547,27007.269531,26786.630859,26941.019531,2176480400
2017-07-27,27002.699219,27169.150391,26962.029297,27131.169922,1763945900
2017-07-28,27007.970703,27047.880859,26928.439453,26979.390625,1456040200
2017-07-31,27048.449219,27324.380859,27045.410156,27323.990234,1693214100
2017-08-01,27326.039062,27558.000000,27326.039062,27540.230469,2392422600


In [5]:
data.shape

(1231, 5)

# generate features

In [7]:
import talib
def features(dataset):
    for i in [5,10,20,60]:
        window = i
        # Volatility
        dataset[str('SMA'+str(i))] = talib.SMA(dataset['Close'],timeperiod=window).shift(1)
        dataset[str('EMA'+str(i))] = talib.EMA(dataset['Close'],timeperiod=window).shift(1)
        dataset[str('STDDEV'+str(i))] = talib.STDDEV(dataset['Close'],timeperiod=window).shift(1)
        dataset[str('Lag'+str(i))] = dataset['Close'].shift(i)
    return dataset

In [8]:
data = features(data)

In [9]:
data = data.iloc[60:,:]
data.index = range(len(data))
data.shape

(1171, 21)

In [10]:
data.head()

,Open,High,Low,Close,Volume,SMA5,EMA5,STDDEV5,Lag5,SMA10,...,STDDEV10,Lag10,SMA20,EMA20,STDDEV20,Lag20,SMA60,EMA60,STDDEV60,Lag60
0,28557.800781,28557.800781,28253.320312,28305.880859,1712055400,28549.676172,28471.317898,212.160632,28692.800781,28489.083008,...,167.568349,28326.589844,28176.189941,28271.007228,424.844184,28110.330078,27793.300846,27793.300846,459.334291,26941.019531
1,28232.619141,28356.099609,28098.830078,28154.970703,1426893600,28472.292187,28416.172219,216.365785,28697.490234,28487.012109,...,169.678410,28490.830078,28185.967480,28274.328527,425.465734,27880.529297,27816.048535,27810.106748,450.270045,27131.169922
2,28238.849609,28387.210938,28198.560547,28302.890625,1272809300,28363.788281,28329.105047,212.218774,28711.759766,28453.426172,...,196.688687,28389.570312,28199.689551,28262.961115,419.781147,27500.339844,27833.111882,27821.413763,443.338224,26979.390625
3,28165.609375,28316.199219,28102.750000,28202.380859,1431611800,28282.014453,28320.366906,121.961974,28159.089844,28444.758203,...,201.170683,28459.029297,28239.817090,28266.763925,388.180389,27513.009766,27855.170215,27837.199890,433.120167,27323.990234
4,28332.910156,28494.519531,28305.730469,28438.849609,2255521100,28290.672656,28281.038224,114.218950,28487.240234,28419.093359,...,213.694340,28476.429688,28274.285645,28260.632205,350.932419,27642.429688,27869.810059,27849.173036,429.750476,27540.230469


# 2. evalution functions

In [11]:
SERIES = pd.Series(data['Close'].values)

In [12]:
# Model evaluation function
def evl(y_test, y_pred): # MSE and MAE are different on different scales
    y_test,y_pred = np.array(y_test).ravel(),np.array(y_pred).ravel()
    r2 = r2_score(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)*100
    print('Model Evaluation ')
    print('R2:', r2)
    print('RMSE:', rmse)
    print('MAE:', mae)
    print("MAPE:",mape) # MAPE before normalization may error beacause of negative values
    return [r2,rmse,mae,mape]

In [13]:
# Plot original data and forecasting data
def plot_all(pred_ans,PERIODS):
    # Check and Change
    pred_ans = pred_ans.ravel()
    og = pd.Series(SERIES[-PERIODS:].values)

    # Plot
    fig = plt.figure(figsize=(10,4))
    
    plt.plot(og,label= 'Original data', color='#0070C0') #F27F19 orange #0070C0 blue
    plt.plot(pred_ans,label= 'Forecasting data', color='#F27F19')
    plt.xlabel('')
    plt.title('forecasting results')
    plt.legend()
              
    plt.show()
    return 

# 3.prediction function

In [14]:
#without scaler
# DATE_BACK functions for inputting sets
def create_dateback(df,DATE_BACK=5,ahead = 1):
    if isinstance(df, pd.DataFrame):
        trainX = df.drop('Close', axis=1, inplace=False)
        #scalarX =MinMaxScaler(feature_range=(0,1))#sklearn normalize
        #trainX = scalarX.fit_transform(trainX)
        trainY = np.array(df['Close']).reshape(-1, 1)
        #scalarY = MinMaxScaler(feature_range=(0,1))#sklearn normalize
        #trainY = scalarY.fit_transform(trainY)
    # Normalize for each IMF in Series
    else:
        trainY = np.array(df.values).reshape(-1, 1)
        #scalarY = MinMaxScaler(feature_range=(0,1))#sklearn normalize
        #trainY = scalarY.fit_transform(trainY)
        trainX = trainY
    
    # Create dateback
    dataX, dataY = [], []
    ahead = ahead - 1
    for i in range(len(trainY)-DATE_BACK-ahead):
        dataX.append(np.array(trainX[i:(i+DATE_BACK)]))
        dataY.append(np.array(trainY[i+DATE_BACK+ahead]))
        
    return np.array(dataX),np.array(dataY)

In [15]:
def step_linear_model(df,model_type,ahead = 1):
    trainX,trainY = create_dateback(df)
    X0_train,X0_test = trainX[:-ahead],trainX[-ahead:]
    y0_train = trainY[:-ahead]
    
    X0_train = X0_train.reshape(X0_train.shape[0],-1)
    X0_test = X0_test.reshape(X0_test.shape[0],-1)
    y0_train = y0_train.reshape(y0_train.shape[0])
    
    #define linear model
    if model_type == 'LR':
        regressor = LinearRegression()
    if model_type =='Lasso':
        regressor = Lasso(alpha=0.1)
    if model_type == 'Ridge':
        regressor = Ridge()
    if model_type == 'Xgboost':
        regressor = XGBR()
    if model_type == 'SVR':
        regressor = SVR()
    
    #print(X0_train.shape)
    #print(y0_train.shape)
    model = regressor.fit(X0_train,y0_train)
    # prediction on test set
    y_pred = model.predict(X0_test)
    
    return y_pred[0]

# bench

## 4.XGBOOST

In [16]:
test_num = int(data.shape[0]*0.2)

# denoising

In [17]:
# 3.Decomposition, Sample entropy, Re-decomposition, and Integration
# EMD decomposition
# -------------------------------
# Decompose adaptively and plot function
# Residue is named the last IMF
#for show
def emd_decom(series,MODE,trials=10,draw=True): 
    # Check input
    series = series.values

    # Initialization
    #print('%s decomposition is running.'%str.upper(MODE))
    if MODE == 'emd':decom = EMD()
    elif MODE == 'eemd':decom = EEMD()
    elif MODE == 'ceemdan':decom = CEEMDAN()
        
    # Decompose
    decom.trials = trials # Number of the white noise input
    imfs_emd = decom(series)
    imfs_num = np.shape(imfs_emd)[0]

    if draw:
        # Plot original data
        series_index = range(len(series))
        fig = plt.figure(figsize=(16,2*imfs_num))
        plt.subplot(1+imfs_num, 1, 1 )
        plt.plot(series_index, series, color='#0070C0') #F27F19 orange #0070C0 blue
        plt.ylabel('Original data')
        
        
        # Plot IMFs
        for i in range(imfs_num):
            plt.subplot(1 + imfs_num,1,2 + i)
            plt.plot(series_index, imfs_emd[i, :], color='#F27F19')
            plt.ylabel(str.upper(MODE)+'-IMF'+str(i))
    
    imfs_df = pd.DataFrame(imfs_emd.T)
    imfs_df.columns = ['imf'+str(i) for i in range(imfs_num)]

    return imfs_df # pd.DataFrame

In [18]:
from PyEMD.checks import whitenoise_check
def emd_denoise(mode,series,show = False):
    if mode == 'emd':
        decom = EMD()
    elif mode == 'eemd':
        decom = EEMD()
    elif mode == 'ceemdan':
        decom = CEEMDAN()
    imfs = decom(series)
    significant_imfs = whitenoise_check(imfs,alpha = 0.95)
    denoised_data = np.array([0]*len(series))
    if show:print(significant_imfs)
    for i in significant_imfs:
        if significant_imfs[i] ==1:
            denoised_data = denoised_data+imfs[i-1]
    return denoised_data,imfs.shape[0]

In [19]:
def denoise_1(de_data,df):
    de_df = df.drop(axis = 1, columns = 'Close',inplace = False)
    de_df['de_close'] = pd.DataFrame(de_data)
    return de_df

In [20]:
#without scaler
# DATE_BACK functions for inputting sets
def df_dateback(df,DATE_BACK=5,ahead = 1):
    trainX = df.drop('de_close', axis=1, inplace=False)
    trainY = np.array(df['de_close']).reshape(-1, 1)
    
    # Create dateback
    dataX, dataY = [], []
    ahead = ahead - 1
    for i in range(len(trainY)-DATE_BACK):
        dataX.append(np.array(trainX[i:(i+DATE_BACK)]))
        dataY.append(np.array(trainY[i+DATE_BACK]))
    dataX.append(np.array(trainX[-DATE_BACK:]))
        
    return np.array(dataX),np.array(dataY)

In [21]:
def denoise_pred(df,model_type,ahead = 1):
    trainX,trainY = df_dateback(df)
    X0_train,X0_test = trainX[:-ahead],trainX[-ahead:]
    y0_train = trainY
    
    X0_train = X0_train.reshape(X0_train.shape[0],-1)
    X0_test = X0_test.reshape(X0_test.shape[0],-1)
    y0_train = y0_train.reshape(y0_train.shape[0])
    
    #define linear model
    if model_type == 'LR':
        regressor = LinearRegression()
    if model_type =='Lasso':
        regressor = Lasso(alpha=0.1)
    if model_type == 'Ridge':
        regressor = Ridge()
    if model_type == 'Xgboost':
        regressor = XGBR()
    if model_type == 'SVR':
        regressor = SVR()
    
    
    #print(X0_train.shape)
    #print(y0_train.shape)
    model = regressor.fit(X0_train,y0_train)
    # prediction on test set
    y_pred = model.predict(X0_test)
    
    return y_pred[0]

# 1.emd

In [23]:
import time

In [25]:
emd_time = 0
for i in range(test_num,test_num-100,-1):
    print(test_num-i)
    series = SERIES[:-i]
    de_df = data[:-i]
    start = time.time()
    denoised_data,n = emd_denoise('emd',series.values)
    end = time.time()
    run_time = end - start
    emd_time  = run_time + emd_time

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [26]:
emd_time

5.587433815002441

# 2. ceemdan

In [28]:
ceemdan_time = 0
for i in range(test_num,test_num-100,-1):
    print(test_num-i)
    series = SERIES[:-i]
    de_df = data[:-i]
    start = time.time()
    denoised_data,n = emd_denoise('ceemdan',series.values,show = False)
    end = time.time()
    run_time = end - start
    ceemdan_time  = run_time + ceemdan_time

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [29]:
ceemdan_time

676.2155845165253

# VMD-DFA

In [30]:
import fathon
from fathon import fathonUtils as fu

In [31]:
def calculate_H(array):
    a = fu.toAggregated(array)
    pydfa = fathon.DFA(a)
    wins = fu.linRangeByStep(4,16)
    n,F = pydfa.computeFlucVec(wins)
    H,H_intercept = pydfa.fitFlucVec()
    return H

In [32]:
def plot_dfa(H_list):
    plt.plot(H_list)
    plt.axhline(0.75,color = 'red')
    plt.xlabel('IMF')
    plt.show()

In [33]:
def J_number(num):
    if num <= 0.75:
        J = 1
    elif num> 0.75 and num <= 1.0:
        J = 2
    elif num > 1 and num <= 1.5:
        J = 3
    else:
        J = 4
    return J

In [34]:
def K_num(series,J):
    K = 1
    num = 0
    while (num < 20):
        num = 0
        vmf_imfs = vmd_decom(series,K,draw=False)
        for i in vmf_imfs.columns:
            if calculate_H(vmf_imfs[i]) >= 0.75:
                num = num +1
        if num > J:
                return K-1
        K = K + 1
    #the Max K
    #K = K -1
    #return K

In [35]:
# VMD # There are some problems in this module
# -------------------------------
def vmd_decom(series,K,draw = False):
    from vmdpy import VMD  
    # VMD parameters
    alpha = 2000       # moderate bandwidth constraint  
    tau = 0.           # noise-tolerance (no strict fidelity enforcement)  
    DC = 0             # no DC part imposed  
    init = 1           # initialize omegas uniformly  
    tol = 1e-7         

    # VMD 
    imfs_vmd, imfs_hat, omega = VMD(series, alpha, tau, K, DC, init, tol)  
    imfs_num = np.shape(imfs_vmd)[0]
    
    if draw:
        # Plot original data
        fig = plt.figure(figsize=(16,2*imfs_num))
        plt.subplot(1+imfs_num, 1, 1 )
        plt.plot(series, color='#0070C0') #F27F19 orange #0070C0 blue
        plt.ylabel('VMD Original data')
    
        # Plot IMFs
        for i in range(imfs_num):
            plt.subplot(1 + imfs_num,1,2 + i)
            plt.plot(imfs_vmd[i, :], color='#F27F19')
            plt.ylabel('VMD-IMF'+str(i))
            
        plt.show()
    
    # Save data
    imfs_df = pd.DataFrame(imfs_vmd.T)
    imfs_df.columns = ['imf'+str(i) for i in range(imfs_num)]

    return imfs_df # pd.DataFrame

In [36]:
def put_databack(series,reconstructed_data,df):
    if len(series) > len(reconstructed_data):
        reconstructed_data = list(reconstructed_data)
        reconstructed_data.insert(0,reconstructed_data[0])

    de_df = df
    de_df.drop(axis = 1, columns = 'Close',inplace = True)
    de_df['de_close'] = pd.DataFrame(reconstructed_data)
    return de_df

In [37]:
#one example
vmd_time = 0
for i in range(test_num,test_num-100,-1):
    print(test_num-i)
    series = SERIES[:-i]
    start = time.time()
    alpha0 = calculate_H(series)
    J = J_number(alpha0)
    K = K_num(series,J)
    vmf_imfs = vmd_decom(series,K,draw=False)
    reconstructed_data = vmf_imfs.iloc[:,:J].sum(axis = 1)
    end = time.time()
    run_time = end - start
    vmd_time  = run_time + vmd_time

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [38]:
vmd_time

113.73246192932129

# VMD-EMD-DFA

In [ ]:
emd_time = 0
for i in range(test_num,test_num-100,-1):
    print(test_num-i)
    series = SERIES[:-i]
    de_df = data[:-i]
    
    denoised_data,n = emd_denoise('emd',series.values)
    end = time.time()
    run_time = end - start
    emd_time  = run_time + emd_time

In [40]:
decom = EMD()
num = decom(series.values).shape[0]
num

8

In [41]:
vmd_emd_time = 0
for i in range(test_num,test_num-100,-1):
    print(test_num-i)
    series = SERIES[:-i]
    
    start = time.time()
    decom = EMD()
    num = decom(series.values).shape[0]
    vmf_imfs = vmd_decom(series,num,draw=False)
    col_num = []
    no = 0
    for j in vmf_imfs.columns:
        if calculate_H(vmf_imfs[j]) >= 0.75:
            col_num.append(no)
        no = no+1
    reconstructed_data = vmf_imfs.iloc[:,col_num].sum(axis = 1)
    end = time.time()
    run_time = end - start
    vmd_emd_time  = run_time + vmd_emd_time

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [42]:
vmd_emd_time

48.030542612075806